# (phase 1) 02 Class overlap quantification and identification

The identification and quantification of class overlap is a key step of the study. In this notebook, overlapping regions/instances are detected using three different methods.


- 1. (Structural Overlap), N1 metric
- 2. (Instance Overlap), Clst. Number of clusters needed to cover the entire data domain, building the ratio between pure, containing instances of the same class, and mixed clusters, which present instances of the two opposite classes.
- 3. Discriminant analysis (SVDD)

## Import libraries, data and visualize overlap

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import importlib
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.cluster import KMeans
from scipy.spatial.distance import squareform
from mpl_toolkits.mplot3d import Axes3D
import gower
import prince
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.metrics import f1_score, make_scorer, confusion_matrix, recall_score, roc_auc_score, roc_curve, average_precision_score
from sklearn.linear_model import LogisticRegression
from collections import Counter
from typing import List
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as imbPipeline
from tqdm import tqdm
from scipy.spatial.distance import euclidean, hamming
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import BallTree
from sklearn.neighbors import KernelDensity
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
from sklearn.svm import OneClassSVM

# import from custom package
from auxFuns.EDA import *
from auxFuns.modelling import *
from auxFuns.class_overlap import *

In [3]:
import auxFuns.EDA 
importlib.reload(auxFuns.EDA)

import auxFuns.modelling
importlib.reload(auxFuns.modelling)

import auxFuns.class_overlap
importlib.reload(auxFuns.class_overlap)

<module 'auxFuns.class_overlap' from 'c:\\Users\\angel\\Documents\\VSCode\\rsv_modelling_transfer_learning\\auxFuns\\class_overlap.py'>

In [4]:
# Load phase 1 data
raw_datasets_path = os.getcwd() + '/datasets/raw'
processed_datasets_path = os.getcwd() + '/datasets/processed'

# Phase 1 data
rsv_predictors_df_v2 = pd.read_csv(processed_datasets_path + '/rsv_predictors_phase1_daysDedup_seasons_prevTest_v2.csv',low_memory=False)
rsv_predictors_phase1_df = make_it_categorical_v2(rsv_predictors_df_v2)

In [5]:
# Following the model selection step, these are the final features taken for modelling
selected_features_v2 = ['n_tests_that_day', 'sine','cosine', 'previous_test_daydiff',
                     'Bronchitis', 'CCI',
                     'Acute_upper_respiratory_infection', 'n_immunodeficiencies', 'n_symptoms',
                     'healthcare_seeking', 
                     'General_symptoms_and_signs', 'prev_positive_rsv', 'Influenza',
                     'key_comorbidities','Pneumonia',
                     'season','month_of_the_test','multiple_tests',
                     'BPA','BPAI']
selected_features_v2.append('RSV_test_result')

df_modelling_phase1 = rsv_predictors_phase1_df[selected_features_v2]

df_modelling_phase1.shape

(86058, 21)

In [ ]:
# Visualization of class overlap
# As the data is high dimensional, it is needed to bring it to a lower dimension for accessible visualization

random_seed = 40
n_components = 5

X = df_modelling_phase1.drop(['RSV_test_result'], axis = 1)

famd = prince.FAMD(n_components=n_components, random_state=random_seed)
famd = famd.fit(X)

df_visualization = famd.transform(X)

# Ensure both df1 and df_transformed present the same records in the same order
df_modelling_phase1 = df_modelling_phase1.sort_index()
df_visualization = df_visualization.sort_index()

assert all(df_modelling_phase1.index == df_visualization.index), "The indices of df1 and df_transformed do not match."

df_visualization['RSV_test_result'] = df_modelling_phase1['RSV_test_result']

In [ ]:
light_red = (0.9, 0.4, 0.4) # RGB for light red
light_green = (0.4, 0.9, 0.4) # RGB for light green
light_blue = (0.4, 0.4, 0.9) # RGB for light blue

plot_5FMDA_planes(df = df_visualization, hue_target = 'RSV_test_result', palette = {'Positive':light_red, 'Negative':light_blue}, figsize = (10,12))

## 1. Structural overlap: N1

In [ ]:
# Calculate the N1 and N1 augmented metric for 1-5 number of neighbours

X_source = pd.get_dummies(X)
labels = df_modelling_phase1.RSV_test_result
same_class_neighbours_dict, N1_metric_dict, dist_matrix, ind = calculate_same_neighbours_and_N1(X = X_source, y = labels, n_neighbours = 6)

In [ ]:
# Augmented N1 > extension of N1 metric to class imbalance

for n_neighbours in range(1,6):   
    same_class_neighbours = same_class_neighbours_dict[n_neighbours]

    print('\n-------')
    print(f'Augmented N1 metric for {n_neighbours} neighbours')
    find_augmented_n1_metric_from_same_neighbours(same_class_neighbours = same_class_neighbours, labels = labels)


In [ ]:
# Visualize the presence of overlapping instances according to the N1 metric
df_visualization_overlapping_regions = df_visualization.copy()
df_visualization_overlapping_regions['overlapping']= ~same_class_neighbours_dict[1]

bright_orange = (1.0, 0.8, 0.2) # RGB for light red
light_light_blue = (0.4, 0.4, 0.6) # RGB for light blue

plot_5FMDA_planes(df = df_visualization_overlapping_regions, hue_target = 'overlapping', palette = {True:bright_orange, False:light_light_blue}, figsize = (10,12))

## 2. Instance-level overlap: Clusters

Per eps_value, we want to obtain the following:
- Number of clusters 
  number of noise points (and as a percentage of the total points)
- number of positives in noise (and as a percentage)
- number of clusters and of points in mixed/positive/negative clusters

In [ ]:
eps_values = [0.05, 0.1, 0.5, 0.7, 0.9, 1, 1.1, 1.3, 1.5]
min_samples_value = 5
# we need to cluster using the famd data, if not it is not possible to build the clusters
random_seed = 40
n_components = 5
X = df_modelling_phase1.drop(['RSV_test_result'], axis = 1)
famd = prince.FAMD(n_components=n_components, random_state=random_seed)
famd = famd.fit(X)
X_famd = famd.transform(X)
df_famd = pd.concat([X_famd, df_modelling_phase1['RSV_test_result']], axis = 1)
results = []


for eps_value in eps_values:
    print('\n----------')
    print(f'Eps value of: {eps_value}')

    # 0: fit dbscan and obtain labels
    db = DBSCAN(eps = eps_value, min_samples = min_samples_value).fit(X_famd) 
    labels = db.labels_
    df_famd['DBSCAN_labels'] = db.labels_

    # 1. Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)
    p_noise = 100*(n_noise_/(X_famd.shape[0]))
    print(f'Estimated number of clusters: {n_clusters_}')
    print(f'Estimated number of noise points: {n_noise_} and percentage over total data domain: {p_noise}')

    # 2. Number of positives in noise (and as a percentage)
    df_famd['is_noise'] = df_famd['DBSCAN_labels'].apply(lambda x: 'Noise' if x == -1 else 'Clustered')
    df_no_noise = df_famd.loc[df_famd['is_noise'] != 'Noise']

    print('-----------')
    total_pos = df_famd['RSV_test_result'].value_counts()[1]
    print(f'Total number of positive records: {total_pos}')
    for c in df_famd['is_noise'].unique():
        cluster_df = df_famd.loc[df_famd['is_noise'] == c,:]
        pos = cluster_df['RSV_test_result'].value_counts()['Positive']
        print(f'Number of positives in {c}: {pos}')

    # 3. Number of clusters and of points in mixed/positive/negative clusters

    count_pos = 0
    count_neg = 0
    count_mix = 0
    for cl in df_no_noise['DBSCAN_labels'].unique():
        # print(f'Cluster {cl}')
        cluster_df = df_famd.loc[df_famd['DBSCAN_labels'] == cl,:]
        # neg = aux_df['RSV_test_result'].value_counts().iloc[0]
        pos = cluster_df.RSV_test_result.value_counts()['Positive']
        neg = cluster_df.RSV_test_result.value_counts()['Negative']

        if neg == 0:
            df_famd.loc[df_famd['DBSCAN_labels'] == cl, 'mixed_or_unique'] = 'Positive'
            count_pos += 1
        elif pos == 0:
            df_famd.loc[df_famd['DBSCAN_labels'] == cl, 'mixed_or_unique'] = 'Negative'
            count_neg += 1
        else:
            df_famd.loc[df_famd['DBSCAN_labels'] == cl, 'mixed_or_unique'] = 'Mixed'
            count_mix += 1
        
        # if len(cluster_df['RSV_test_result'].value_counts()) > 1:
        #     pos = cluster_df.RSV_test_result.value_counts()['Positive']
        #     neg = cluster_df.RSV_test_result.value_counts()['Negative']
        #     pos_to_neg = pos / neg
        #     # print(f'Positive to negative ratio: {pos_to_neg}')
        #     count_mix += 1
        #     df_famd.loc[df_famd['DBSCAN_labels'] == cl, 'mixed_or_unique'] = 'Mixed'

        # else: 
        #     dominant_label = aux_df['RSV_test_result'].value_counts().index[0]
        #     if dominant_label == 1:
        #         # print('Fully positive')
        #         df_famd.loc[df_famd['DBSCAN_labels'] == cl, 'mixed_or_unique'] = 'Positive'
        #         count_pos += 1
        #     elif dominant_label == 0:
        #         # print('Fully negative')
        #         df_famd.loc[df_famd['DBSCAN_labels'] == cl, 'mixed_or_unique'] = 'Negative'
        #         count_neg += 1
    print('-------------')
    print('Results of DBSCAN clustering:')
    print(f'\n# fully positive clusters: {count_pos}')
    if count_pos != 0:
        pos_obs = df_famd['mixed_or_unique'].value_counts()['Positive']
    else: 
        pos_obs = 0
    print(f'observacions on fully positive: {pos_obs}')

    print(f'\n# fully negative clusters: {count_neg}')
    if count_neg != 0:
        neg_obs = df_famd['mixed_or_unique'].value_counts()['Negative']
    else: 
        neg_obs = 0
    print(f'observacions on fully negative: {neg_obs}')

    print(f'\n# mixed clusters: {count_mix}')
    if count_mix != 0:
        mix_obs = df_famd['mixed_or_unique'].value_counts()['Mixed']
    else: 
        mix_obs = 0
    print(f'observacions on mixed clusters: {mix_obs}')

    results.append({
        'eps_value': eps_value,
        'n_clusters_': n_clusters_,
        'n_noise_': n_noise_,
        'count_pos': count_pos,
        'pos_obs': pos_obs,
        'count_neg': count_neg,
        'neg_obs': neg_obs,
        'count_mix': count_mix,
        'mix_obs': mix_obs
    })

results_df = pd.DataFrame(results)


## 3. Structural level: SVD

- 1. Bring the data to a 3-D representation (and split in training and test)
- 2. Train a SVD model in the training data to detect overlapping regions.
- 3. Assign testing points to the overlapping or non-overlappintg

In [7]:
# 1. Bring data to a 3-D representation and split in training and test 
random_seed = 40
n_components = 3

X = df_modelling_phase1.drop(['RSV_test_result'], axis = 1)

famd = prince.FAMD(n_components=n_components, random_state=random_seed)
famd = famd.fit(X)
X_3d = famd.transform(X)

# train test split
labels = df_modelling_phase1.RSV_test_result

X_train, X_validation, y_train, y_validation = train_test_split(X_3d, labels, test_size=0.2, random_state=random_seed, stratify = labels)

In [8]:
# 2. Train a SVD to find the overlapping regions in the training datadf_
X_training_positive = X_train.loc[labels == 'Positive',:]
X_training_negative = X_train.loc[labels == 'Negative',:]

# Train One-Class SVM for positive and negative samples, separately.
print('Fitting positive oneSVM...')
ocsvm_positive = OneClassSVM()
ocsvm_positive.fit(X_training_positive)

print('Fitting negative oneSVM...')
ocsvm_negative = OneClassSVM()
ocsvm_negative.fit(X_training_negative)

# Distances for positive and negative datasets to their specific boundary
print('Finding distances to boundary ...')
distances_positive = ocsvm_positive.decision_function(X_training_positive)
distances_negative = ocsvm_negative.decision_function(X_training_negative)

# Find overlaps 
# Thereshold of 1 e-03 can be tuned
print('Finding overlapping points...')
threshold = 40
overlapping_positive_mask = np.abs(distances_positive) < threshold
overlapping_negative_mask = np.abs(distances_negative) < threshold
all_overlapping_mask = np.concatenate([overlapping_positive_mask, overlapping_negative_mask], axis = 0)

overlapping_positive_mask.sum(), overlapping_negative_mask.sum(), all_overlapping_mask.sum()

# Reorder the X_train and y_train to ensure the overlapping has been correctly identified 
X_training = pd.concat([X_training_positive, X_training_negative], axis = 0)
y_training = pd.concat([y_train[y_train == 'Positive'], y_train[y_train == 'Negative']], axis = 0)

Fitting positive oneSVM...
Fitting negative oneSVM...
Finding distances to boundary ...
Finding overlapping points...


In [9]:
# Find overlaps in the training set
# Thereshold of 1 e-03 can be tuned
threshold = 20
overlapping_positive_mask = np.abs(distances_positive) < threshold
overlapping_negative_mask = np.abs(distances_negative) < threshold
all_overlapping_mask = np.concatenate([overlapping_positive_mask, overlapping_negative_mask], axis = 0)
print(overlapping_positive_mask.sum(), overlapping_negative_mask.sum())

overlapping_positive_mask.sum(), overlapping_negative_mask.sum(), all_overlapping_mask.sum()

# Reorder the X_train and y_train to ensure the overlapping has been correctly identified 
X_training = pd.concat([X_training_positive, X_training_negative], axis = 0)
y_training = pd.concat([y_train[y_train == 'Positive'], y_train[y_train == 'Negative']], axis = 0)

788 1527


In [ ]:
light_red = (0.9, 0.4, 0.4)
light_blue = (0.4, 0.4, 0.9)

def plot_contours_and_points(ax, axis1, axis2, Z_positive_norm, Z_negative_norm, X_training_positive, X_training_negative):
    """Plot contours and points on the given axes."""
    ax.contour(xx, yy, Z_positive_norm, levels=[-1, 0, 1], linewidths=2, colors='r')
    ax.contour(xx, yy, Z_negative_norm, levels=[-1, 0, 1], linewidths=2, colors='b')
    ax.scatter(X_training_positive[axis1], X_training_positive[axis2], c=light_red, s=1, label='Positive')
    ax.scatter(X_training_negative[axis1], X_training_negative[axis2], c=light_blue, s=1, label='Negative')
    
    ax.set_title(f"2D Projection ({axis1}, {axis2})")
    ax.set_xlabel(axis1)
    ax.set_ylabel(axis2)
    ax.legend()

x_span = np.linspace(-20, 20, 50) 
y_span = np.linspace(-20, 20, 50)  
xx, yy = np.meshgrid(x_span, y_span)
grid_2d = np.c_[xx.ravel(), yy.ravel()]

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axis_combinations = [(0, 1), (0, 2), (1, 2)]

for i, (axis1, axis2) in enumerate(axis_combinations):
    constant_value = np.zeros(grid_2d.shape[0])
    grid_3d = np.zeros((grid_2d.shape[0], 3))
    grid_3d[:, axis1] = grid_2d[:, 0]
    grid_3d[:, axis2] = grid_2d[:, 1]
    grid_3d[:, 3 - axis1 - axis2] = constant_value  # third dimension filled with constants

    Z_positive = ocsvm_positive.decision_function(grid_3d).reshape(xx.shape)
    Z_negative = ocsvm_negative.decision_function(grid_3d).reshape(xx.shape)
    
    Z_positive_norm = (Z_positive - np.mean(Z_positive)) / np.std(Z_positive)
    Z_negative_norm = (Z_negative - np.mean(Z_negative)) / np.std(Z_negative)
    
    plot_contours_and_points(axes[i], axis1, axis2, Z_positive_norm, Z_negative_norm, X_training_positive, X_training_negative)

plt.suptitle("2D Projections of 3D One-Class SVM Boundaries")
plt.show()

In [15]:
# 3. Assign validation data points to overlapping or non overlapping
# Find the distances of every validation point to the positive and negative boundaries
distances_validation_positive = ocsvm_positive.decision_function(X_validation)
distances_validation_negative = ocsvm_positive.decision_function(X_validation)

overlapping_validation = []

# Assign labels based on boundaries
# threshold = 0.1  # Define a threshold for determining overlapping points
for dist_pos, dist_neg in zip(distances_validation_positive, distances_validation_negative):
    if np.abs(dist_pos) < threshold and np.abs(dist_neg) < threshold:
        overlapping_validation.append('Overlapping')
    elif np.abs(dist_pos) < threshold:
        overlapping_validation.append('Likely Positive')
    elif np.abs(dist_neg) < threshold:
        overlapping_validation.append('Likely Negative')
    else:
        overlapping_validation.append('Non-overlapping')


In [122]:
# 4. Now, see if this helps in the prediction
## 4.1. Fit the overlapping and non overlapping models in the two distinct regions
print('\n------------------')
print('-------------------')
print('Fitting model on trainig data exclusively')

model_nonOverlapping, model_Overlapping = only_build_2overlapping_models(X = X_training, labels = y_training, is_overlapping = all_overlapping_mask, 
                                                                        random_seed = 42,
                                                                        model_class_non_overlapping = RandomForestClassifier(),
                                                                        param_grid_non_overlapping = {'n_estimators': [7, 14],'max_depth': [10, 20],'min_samples_split': [5, 10],'min_samples_leaf': [1, 4]},
                                                                        cost_sensitive_non_overlapping = False, weight_dict_non_overlapping = {'Negative': 1, 'Positive': 15},
                                                                        model_class_overlapping = RandomForestClassifier(),
                                                                        param_grid_overlapping = {'n_estimators': [7, 14],'max_depth': [10, 20],'min_samples_split': [5, 10],'min_samples_leaf': [1, 4]},
                                                                        cost_sensitive_overlapping = False, weight_dict_overlapping = {'Negative': 1, 'Positive': 15})


------------------
-------------------
Fitting model on trainig data exclusively
----------------
Building non-overlapping model ...
Training model ... RandomForestClassifier(random_state=42)
Best training parameters:  {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 7}
Best training f1-score:  0.01712885621846345
----------------
Building (yes) overlapping model ...
Training model ... RandomForestClassifier(random_state=42)
Best training parameters:  {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 14}
Best training f1-score:  0.8852457149927364
